## Exmple using imported van der Weilen et al gocad model for Emmie Bluff region


In [ ]:
from LoopFlow import import_triangulation as it
import pandas as pd
import numpy as np
from math import degrees,atan,asin,sqrt,pow, atan2
from os import listdir
from os.path import isfile, join
import networkx as nx
import os
import shutil
from LoopFlow import calculate_flow as cf
from LoopStructural.modelling import ProcessInputData, Map2LoopProcessor
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer
from LoopStructural.datasets import load_geological_map_data
import matplotlib.pyplot as plt
from datetime import datetime
from LoopFlow import __version__
print(__version__)

In [ ]:
destination = "./Emmie_Bluff_" +(datetime.now()).strftime('%Y%m%d-%H%M%S')
if(not os.path.isdir(destination)):
    os.mkdir(destination)
root_dir=destination
out_dir=destination
suffix='.ts'
tsurfs='../tsurfs/'
strat_contact_frac=0.25

In [ ]:
contacts_points,stratigraphic_orientations,stratigraphic_thickness,stratigraphic_order,fault_orientations,fault_edges,fault_properties,fault_locations,bbox=it.import_triangles(tsurfs,suffix,strat_contact_frac)

In [ ]:
fault_properties

In [ ]:
if(not os.path.isdir(destination+'/dtm')):
    os.mkdir(destination+'/dtm')
if(not os.path.isdir(destination+'/vtk')):
    os.mkdir(destination+'/vtk')

#get dtm
import rasterio
from shapely.geometry import Point
import geopandas as gpd
from owslib.wcs import WebCoverageService
from rasterio.warp import calculate_default_transform, reproject, Resampling

try:
    shutil.copyfile(obj_path_dir+'dtm_rp.tif', out_dir + '/dtm/'+'dtm_rp.tif')
except:
    def get_dtm(path_out, minlong, maxlong, minlat, maxlat,
                url="http://services.ga.gov.au/gis/services/DEM_SRTM_1Second_over_Bathymetry_Topography/MapServer/WCSServer?"
                ):

        bbox = (minlong, minlat, maxlong, maxlat)

        wcs = WebCoverageService(url, version='1.0.0')

        cvg = wcs.getCoverage(identifier='1',  bbox=bbox,
                              format='GeoTIFF', crs=4326, width=200, height=200)

        f = open(path_out, 'wb')
        bytes_written = f.write(cvg.read())
        f.close()
        print("dtm geotif saved as", path_out) 
        
    def reproject_dtm(path_in, path_out, src_crs, dst_crs):
        with rasterio.open(path_in) as src:
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            with rasterio.open(path_out, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
                dst.close()
        print("reprojected dtm geotif saved as", path_out)
        
    corner_pts = {'col1': ['bl', 'tr'], 'geometry': [Point(bbox.minx,bbox.miny), Point(bbox.maxx,bbox.maxy)]}    
    corner_pts_gpd=gpd.GeoDataFrame(corner_pts ,crs='epsg:28353' )  
    corner_pts_gpd=corner_pts_gpd.to_crs('epsg:4326')
    #display(corner_pts_gpd)
    
    get_dtm(out_dir+ '/dtm/'+'dtm.tif', corner_pts_gpd.loc[0].geometry.x-.1, corner_pts_gpd.loc[1].geometry.x+.1, corner_pts_gpd.loc[0].geometry.y-.1, corner_pts_gpd.loc[1].geometry.y+.1,
                url="http://services.ga.gov.au/gis/services/DEM_SRTM_1Second_over_Bathymetry_Topography/MapServer/WCSServer?"
                )
    reproject_dtm(out_dir+ '/dtm/'+'dtm.tif', out_dir + '/dtm/'+'dtm_rp.tif', 'epsg:4326', 'epsg:28353')

In [ ]:
# Define project pathing from m2l
proj_path = out_dir
graph_path = out_dir+'/graph/'
tmp_path = out_dir+'/tmp/'
data_path = out_dir+'/output/'
dtm_path = out_dir+'/dtm/'
output_path = out_dir+'/output/'
vtk_path = out_dir+'/vtk/'


extra_depth=0

# Define project bounds
bbox=it.create_bbox(bbox.iloc[0].minx,bbox.iloc[0].miny,bbox.iloc[0].maxx,bbox.iloc[0].maxy-15000,bbox.iloc[0].lower,bbox.iloc[0].upper)





minx,miny,maxx,maxy = [bbox.iloc[0].minx,bbox.iloc[0].miny,bbox.iloc[0].maxx,bbox.iloc[0].maxy]
model_base =bbox.iloc[0].lower-extra_depth #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
model_top = bbox.iloc[0].upper


bbox

In [ ]:
stratigraphic_order

In [ ]:
#basements = {'group number': [0,0], 'order': [5,1.5],'unit name':['basement','ubasement'],'group':['all_same','all_same'],'supergroup':['supergroup_0','supergroup_1']}
#basements_df = pd.DataFrame(data=basements)
stratigraphic_order['order']=stratigraphic_order.index
#stratigraphic_order=pd.concat([stratigraphic_order,basements_df])
stratigraphic_order=stratigraphic_order.sort_values(by='order')
stratigraphic_order=stratigraphic_order.reset_index()
stratigraphic_order['order']=stratigraphic_order.index

stratigraphic_order

In [ ]:
thicknesses = dict(
    zip(
        list(stratigraphic_thickness["name"]),
        list(stratigraphic_thickness["thickness"]),
    )
)
#thicknesses['basement']=10000.0
#thicknesses['ubasement']=100.0
thicknesses

In [ ]:
##############################
# Bounding box
# ~~~~~~~~~~~~
# * Origin - bottom left corner of the model # * Maximum - top right hand corner of the model


#origin = xbbox.loc["origin"].to_numpy()  # np.array(bbox[0].split(',')[1:],dtype=float)
#maximum = xbbox.loc["maximum"].to_numpy()  # np.array(bbox[1].split(',')[1:],dtype=float)


origin = np.array([bbox.iloc[0].minx,bbox.iloc[0].miny,bbox.iloc[0].lower])
maximum = np.array([bbox.iloc[0].maxx,bbox.iloc[0].maxy,bbox.iloc[0].upper])
bbox,maximum-origin

In [ ]:
##############################
# Stratigraphic column
# ~~~~~~~~~~~~~~~~~~~~
# The order of stratrigraphic units is defined a list of tuples containing the name of the group and the order of units within the group. For example there are 7 units in the following example that form two groups.

# example nested list
#stratigraphic_order
[                 
    ("youngest_group", ["unit1", "unit2", "unit3", "unit4"]),
    ("older_group", ["unit5", "unit6", "unit7"]),
]


#order = [("supergroup_0", list(stratigraphic_order["unit name"]))]
order = [("supergroup_0", list(stratigraphic_order["unit name"][:3])),("supergroup_1", list(stratigraphic_order["unit name"][3:])),]

In [ ]:
from LoopStructural.visualisation.vtk_exporter import VtkExporter

##############################
# Building a stratigraphic model
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# A ProcessInputData onject can be built from these datasets using the argument names. A full list of possible arguments can be found in the documentation.

#thicknesses['ubasement']=300.0

processor = ProcessInputData(
    contacts=contacts_points,
    contact_orientations=stratigraphic_orientations.rename(
        {"formation": "name"}, axis=1
    ),
    thicknesses=thicknesses,
    stratigraphic_order=order,
    origin=origin,
    maximum=maximum,
)

##############################
# The process input data can be used to directly build a geological model

model = GeologicalModel.from_processor(processor)
model.update()

##############################
# Or build directly from the dataframe and processor attributes.

"""model2 = GeologicalModel(processor.origin, processor.maximum)
model2.data = processor.data
model2.create_and_add_foliation("supergroup_0")
model2.create_and_add_foliation("supergroup_1")
model2.update()
"""

In [ ]:
##############################
# Adding faults
# ~~~~~~~~~~~~~


processor = ProcessInputData(
    contacts=contacts_points,
    contact_orientations=stratigraphic_orientations.rename(
        {"formation": "name"}, axis=1
    ),
    thicknesses=thicknesses,
    stratigraphic_order=order,
    origin=origin,
    maximum=maximum,
    #fault_edges=fault_edges,
    fault_orientations=fault_orientations,
    fault_locations=fault_locations,
    fault_properties=fault_properties,
)

model = GeologicalModel.from_processor(processor)
model.update()

view = LavaVuModelViewer(model)
#view.nelements = 1e8
view.add_model_surfaces()
view.rotation = [-37.965614318847656, 13.706363677978516, 3.110347032546997]
view.interactive()

"""    fault_edges=[('Fault_Surface_01_Final','Fault_Surface_03_Final',
                  'Fault_Surface_01_Final','Fault_Surface_04_Final',
                  'Fault_Surface_02_Final','Fault_Surface_04_Final',
                  'Fault_Surface_02_Final','Fault_Surface_05_Final')
                ],
    
    fault_edge_properties=[{'angle':90},{'angle':90},{'angle':90},{'angle':90}]
"""


In [ ]:
#view.interactive()

In [ ]:
from map2loop.m2l_utils import save_dtm_mesh, save_dtm_ascii, save_parameters


surface_verts = {}
#vtk_path='../exported/vtk/'
model_name=''
colour_path=''
filename='surface_name_{}.vtk'


def function(xyz,tri,name):
    xyz = np.copy(xyz)
    tri = np.copy(tri)
    nanmask = np.all(~np.isnan(xyz),axis=1)
    vert_idx = np.zeros(xyz.shape[0],dtype=int) -1
    vert_idx[nanmask] = np.arange(np.sum(nanmask))
    tri[:] = vert_idx[tri]
    tri = tri[np.all(tri > -1,axis=1),:]
    xyz = xyz[nanmask,:]
    surface_verts[name] = (xyz,tri)

def mask(xyz): # for clipping strat to surface dtm
    from map2loop.map import MapUtil
    import rasterio
    import os
    dtm_map = MapUtil(proj.config.bbox_3d,dtm=rasterio.open(os.path.join(dtm_path,'dtm_rp.tif')))
    xyz=model.rescale(xyz,inplace=False)
    dtmv = dtm_map.evaluate_dtm_at_points((xyz[:,:2]))
    return xyz[:,2]<dtmv

save_dtm_mesh(dtm_path,vtk_path+model_name+'/')

view = VtkExporter(model,vtk_path+model_name+'/')

view.nsteps = np.array([200,200,200])

view.nelements = 1e5
view.add_model_surfaces(function=function,filename=filename,faults=False)
view.nelements=1e5
view.add_model_surfaces(function=function,filename=filename,strati=False,displacement_cmap = 'rainbow')


for layer in surface_verts:
    f=open(vtk_path+model_name+'/'+layer.replace("_iso_0.000000","")+'.obj','w')
    vert=surface_verts[layer][0].shape[0]
    tri=surface_verts[layer][1].shape[0]
    print(layer,vert,tri)
    for v in range(0,vert):
        ostr = "v {} {} {}\n"\
            .format(surface_verts[layer][0][v][0],surface_verts[layer][0][v][1],surface_verts[layer][0][v][2])
        f.write(ostr)
    for t in range(0,tri):
        ostr = "f {} {} {}\n"\
                .format(surface_verts[layer][1][t][0]+1,surface_verts[layer][1][t][1]+1,surface_verts[layer][1][t][2]+1)
        f.write(ostr)
    f.close()    



import pandas as pd
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from geoh5py.objects import BlockModel
from geoh5py.workspace import Workspace
from geoh5py.objects import Surface

def hextofloats(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return tuple(int(h[i:i + 2], 16) / 255. for i in (1, 3, 5))  # skip '#'

def geoh5_create_surface_data(obj_path_dir,colour_path):

    h5file_path = obj_path_dir+"/loop.geoh5"

    workspace = Workspace(h5file_path)
    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    colour_index=0
    #all_sorts = pd.read_csv(os.path.join(colour_path, 'all_sorts_clean.csv'), ",")
    #all_sorts=all_sorts.set_index('code')
    #colour_map=open(obj_path_dir+'/loop_colour_map.clr','w')
    #olour_map.write('{\tStart\tRed\tGreen\tBlue\t}\n')

    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,delimiter=' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
            if(len(i)>0 and len(v)>0):
                # Create a geoh5 surface
                surface = Surface.create(
                          workspace, name=file.replace('.obj',''), vertices=v, cells=i
                            )
                if('Fault_' in file or 'dtm' in file):
                    colours=np.ones(surface.n_cells)*99
                else:
                    colours=np.ones(surface.n_cells)*colour_index
                    rgb=[128,128,128]
                    #colour_map.write('{}\t{}\t{}\t{}\n'.format(colour_index,rgb[0],rgb[1],rgb[2]))
                    #colour_index=colour_index+1
                    

                surface.add_data({
                    "colour_index": {
                        "association":"CELL",
                        "values": colours
                    }
                })
                
                workspace.save_entity(surface)
    workspace.close()
                
    #colour_map.write('{}\t{}\t{}\t{}\n'.format(99,1,1,1))
    #colour_map.close()
    #print("colour map saved as:",obj_path_dir+'/loop_colour_map.clr')


obj_path_dir= vtk_path+model_name+'/'   # directory of existing obj surfaces
colour_path=tmp_path+'/'
geoh5_create_surface_data(obj_path_dir,colour_path)

In [ ]:
voxel_size=750
faults_only=False
                    
scenario={     #free-form scenario   
        'fault_node':1000.0,
        'geological_formation_slow':1000.0,
        'geological_formation_fast':1.0,
        'interformation_node':1000.0,

        'fault_formation':1000.0,
        'same_fault':1000.0,
        'fault_fault':1000.0,
        'interform_fault':1000.0,
        'interform_formation':100.0,
        'interform_interform':100.0,
        'same_interform':100.0,

        'fast_formation_code':['0']
        }
scenario='fast_faults' #pre-defined scenario
                        # fast_faults    
                        # fast_strat_contacts 
                        # fast_both  
                        # fault_barriers_not_paths  
                        # fault_barriers_but_paths_and_fast_strat
                        #homogeneous

source='west' # 'west','north', 'south', top', 'base', 'deep_line', 'point'
target='east' # 'west','north', 'south', top', 'base', 'deep_line', 'point'
fast_formation_code=['0']

ptx=(bbox.loc[0].minx+bbox.loc[0].maxx)/2
pty=(bbox.loc[0].miny+bbox.loc[0].maxy)/2
ptz=bbox.loc[0].lower

f=open(destination+'/parameters.txt','w')
f.write('faults_only = {}\nscenario = {}\nsource = {}\nfast_litho = {}\npoint = {},{},{}\n'.format(faults_only,str(scenario),source,fast_formation_code,ptx,pty,ptz))
f.close()

Graw,df_nodes_raw,df_edges_raw=cf.graph_from_model(model,voxel_size,bbox,destination)

G,scenario=cf.assign_weights(Graw,scenario,source,target,fast_formation_code,faults_only,bbox,ptx,pty,ptz)

In [ ]:
#hardwire dykes as barriers
length_scale_max=857.1428571428569
for e in G.edges:
    if('fault' in G.edges[e]['type']):
        bits=G.edges[e]['label'].split(' - ')
        if('fault' in bits[0]):
            if(int(bits[0].replace('fault',''))>4): #faults 0-4 are 'real' faults
                scale=G.edges[e]['length']/length_scale_max
                G.edges[e]['weight']=1000.0*scale
                G.edges[e]['capacity']=1/(1000.0*scale)
        if('fault' in bits[1]):
            if(int(bits[1].replace('fault',''))>4): #faults 0-4 are 'real' faults
                scale=G.edges[e]['length']/length_scale_max
                G.edges[e]['weight']=1000.0*scale
                G.edges[e]['capacity']=1/(1000.0*scale)


In [ ]:
voxet_df,distance,path=cf.calculate_dist(G,df_nodes_raw,voxel_size,bbox,scenario,destination)
cf.calculate_paths(path,df_nodes_raw,scenario,destination)
scenery=cf.calculate_scenery(G,model,df_nodes_raw,path,scenario,destination)    
cf.merge_outputs(voxet_df,df_nodes_raw,scenery,scenario,destination)

In [ ]:
#cf.save_edges(df_nodes_raw,df_edges_raw,scenario,destination)
#cf.save_nodes(df_nodes_raw,scenario,destination)

In [ ]:
sourcen=-1
targetn=-2
cf.calc_boykov_kolmogorov(G,sourcen,targetn,df_nodes_raw,df_edges_raw,scenario,destination)